In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

# Import data

In [2]:
df_train = pd.read_parquet("../data/processed/train_cleaned.parquet")
df_valid = pd.read_parquet("../data/processed/valid_cleaned.parquet")
df_test  = pd.read_parquet("../data/processed/test_cleaned.parquet")

In [4]:
TEXT_COL = "text_aggressive"

X_train = df_train[TEXT_COL]
X_valid = df_valid[TEXT_COL]
X_test  = df_test[TEXT_COL]

# Encoding

In [9]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=10000,
    min_df=5,
    max_df=0.95,
    sublinear_tf=True,
    norm="l2"
)

In [11]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_valid_tfidf = tfidf.transform(X_valid)
X_test_tfidf  = tfidf.transform(X_test)

# Training

In [12]:
def compute_scale_pos_weight(y: pd.Series) -> float:
    n_pos = (y == 1).sum()
    n_neg = (y == 0).sum()
    return n_neg / max(n_pos, 1)

In [13]:
LABELS = [
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_hate"
]

models = {}

for label in LABELS:
    print(f"Training {label}")

    y_train = df_train[label]
    y_valid = df_valid[label]

    spw = compute_scale_pos_weight(y_train)

    model = XGBClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="logloss",
        scale_pos_weight=spw,
        tree_method="hist",
        n_jobs=-1,
        random_state=42
    )

    model.fit(
        X_train_tfidf,
        y_train,
        eval_set=[(X_valid_tfidf, y_valid)],
        verbose=False
    )

    models[label] = model

Training toxic
Training severe_toxic
Training obscene
Training threat
Training insult
Training identity_hate


# Prediction

In [14]:
def predict_multilabel(models, X):
    y_pred = {}
    y_proba = {}

    for label, model in models.items():
        y_proba[label] = model.predict_proba(X)[:, 1]
        y_pred[label] = (y_proba[label] >= 0.5).astype(int)

    return y_pred, y_proba

# Evaluation

In [27]:
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    hamming_loss,
    classification_report
)

def evaluate_multilabel(
    y_true_df,
    y_pred_dict,
    labels,
    split_name="test",
    output_path="multilabel_results.xlsx"
):
    Y_true = y_true_df[labels].values
    Y_pred = np.column_stack([y_pred_dict[l] for l in labels])

    # ---------- Global metrics ----------
    global_metrics = {
        "split": split_name,
        "macro_f1": f1_score(Y_true, Y_pred, average="macro", zero_division=0),
        "micro_f1": f1_score(Y_true, Y_pred, average="micro", zero_division=0),
        "exact_accuracy": accuracy_score(Y_true, Y_pred),
        "hamming_loss": hamming_loss(Y_true, Y_pred),
    }

    df_global = pd.DataFrame([global_metrics])

    # ---------- Per-label detailed metrics ----------
    rows = []

    for i, label in enumerate(labels):
        report = classification_report(
            Y_true[:, i],
            Y_pred[:, i],
            output_dict=True,
            zero_division=0
        )

        for cls, metrics in report.items():
            if cls == "accuracy":
                continue

            rows.append(
                {
                    "split": split_name,
                    "label": label,
                    "class": cls,
                    "precision": metrics.get("precision"),
                    "recall": metrics.get("recall"),
                    "f1_score": metrics.get("f1-score"),
                    "support": metrics.get("support"),
                }
            )

    df_details = pd.DataFrame(rows)

    # ---------- Save to Excel (no explicit engine) ----------
    with pd.ExcelWriter(output_path) as writer:
        df_global.to_excel(writer, sheet_name="global_metrics", index=False)
        df_details.to_excel(writer, sheet_name="per_label_metrics", index=False)

    print(f"Results saved to {output_path}")

    return df_global, df_details

In [ ]:
# TRAIN
y_pred_train, _ = predict_multilabel(models, X_train_tfidf)
evaluate_multilabel(df_train, y_pred_train, LABELS, "train", "../reports/results/results_train.xlsx")

# VALID
y_pred_valid, _ = predict_multilabel(models, X_valid_tfidf)
evaluate_multilabel(df_valid, y_pred_valid, LABELS, "valid", "../reports/results/results_valid.xlsx")  
# TEST
y_pred_test, _ = predict_multilabel(models, X_test_tfidf)
evaluate_multilabel(df_test, y_pred_test, LABELS, "test", "../reports/results/results_test.xlsx")

Results saved to ../data/results/results_train.xlsx
Results saved to ../data/results/results_valid.xlsx
Results saved to ../data/results/results_test.xlsx


(  split  macro_f1  micro_f1  exact_accuracy  hamming_loss
 0  test  0.485321  0.620728        0.869909      0.035468,
    split          label         class  precision    recall  f1_score  support
 0   test          toxic             0   0.973178  0.961565  0.967337  14414.0
 1   test          toxic             1   0.677156  0.752591  0.712883   1544.0
 2   test          toxic     macro avg   0.825167  0.857078  0.840110  15958.0
 3   test          toxic  weighted avg   0.944537  0.941346  0.942717  15958.0
 4   test   severe_toxic             0   0.997854  0.970767  0.984124  15804.0
 5   test   severe_toxic             1   0.207547  0.785714  0.328358    154.0
 6   test   severe_toxic     macro avg   0.602700  0.878241  0.656241  15958.0
 7   test   severe_toxic  weighted avg   0.990227  0.968981  0.977796  15958.0
 8   test        obscene             0   0.992826  0.978590  0.985657  15133.0
 9   test        obscene             1   0.689060  0.870303  0.769148    825.0
 10  test   